In [1]:
from braket.aws import AwsDevice
from braket.aws import AwsQuantumJob, AwsSession
from braket.jobs.image_uris import Framework, retrieve_image
from braket.jobs.config import InstanceConfig

import boto3

import os
import json
import time

In [2]:
import copy

c = copy.deepcopy

def get_key(single_dict):
    for k in single_dict.keys():
        return k

def parse_params(params_list, hp, hp_list):
    params = params_list[0]
    k = get_key(params)
    ps = params[k]
    for p in ps:
        hp[k] = p
        if len(params_list) == 1:
            hp_list.append(c(hp))
        else:
            parse_params(params_list[1:], hp, hp_list)

def get_quantum_device(device_name):
    device_arn = "arn:aws:braket:::device/quantum-simulator/amazon/sv1"
    try:
        device = AwsDevice.get_devices(names=[device_name])
        device_arn = device[0].arn
    except Exception as e:
        print(f"fail to get {device_name}: {e}, use sv1 instead")
    return device_arn

def upload_data(dir_name, aws_session=AwsSession()):
    stream_s3_uri = aws_session.construct_s3_uri(aws_session.default_bucket(), dir_name)
    return_path = None
    
    def _check_upload(file_name, check_list):
        file_end = file_name.split('.')[-1]
        if file_end in check_list:
            path = f"{stream_s3_uri}/" + file_name.split('/')[-1]
            aws_session.upload_to_s3(file_name, path)
            
    if os.path.isdir(dir_name):
        dir_list = os.listdir(dir_name)
        for file_name in dir_list:
            _check_upload(os.path.join(dir_name,file_name), ['mol2'])
        return_path = stream_s3_uri
    else:
        _check_upload(file_name, ['mol2'])
        single_file_name = file_name.split('/')[-1]
        return_path = f"{stream_s3_uri}/{single_file_name}"
        
    return return_path

In [29]:
# parameters for experiments
experiment_name = "molecular-unfolding"
data_path = "molecular-unfolding-data"
experiments_params =  {
    "version": "1",
    "params": [
        {"M": [1]},
        {"D": [8]},
        {"shots": [10000]},
        {"device": [{"qc": "null", "cc": "ml.m5.large"}]}
    ]
}

hp = {}
hybrid_job_params = []
parse_params(experiments_params['params'], hp, hybrid_job_params)

print(f"parameters for experiments: \n {hybrid_job_params}")

parameters for experiments: 
 [{'M': 1, 'D': 8, 'shots': 10000, 'device': {'qc': 'null', 'cc': 'ml.m5.large'}}]


In [4]:
# Upload dataset to S3
s3_path = upload_data(data_path)
print(f"upload data to s3 path: {s3_path}")

upload data to s3 path: s3://amazon-braket-us-east-1-002224604296/molecular-unfolding-data


In [5]:
account_id = boto3.client("sts").get_caller_identity()["Account"]
region = boto3.client('s3').meta.region_name
image_name = f"amazon-braket-{experiment_name}-jobs"
image_uri = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{image_name}:latest"

print(f"the hybrid job image for {account_id} in region {region}: {image_uri}")

# # Uncomment the following code if you want to rebuild the container
# !sh build_and_push.sh {image_name}

the hybrid job image for 002224604296 in region us-east-1: 002224604296.dkr.ecr.us-east-1.amazonaws.com/amazon-braket-molecular-unfolding-jobs:latest


In [37]:
# parse evaluation parameters and trigger hybrid jobs:
jobs = []
names = []

job_name = f"{experiment_name}-job"

# from braket.jobs.local import LocalQuantumJob

for job_param in hybrid_job_params:
    M = job_param['M']
    D = job_param['D']
    quantum_device = get_quantum_device(job_param['device']['qc'])
    classical_device = job_param['device']['cc']
    
    print(f"Creating {experiment_name} with M {M} and D {D}")
    name = f"{experiment_name}-M-{M}-D-{D}-" + str(int(time.time()))
    
    # print(f"parameters for quantum job: \n \
    # quantu_device {quantum_device} \n \
    # experiment_name {experiment_name} \n \
    # entry_point {experiment_name}.{job_name}:main \n \
    # name {name} \n \
    # job_param {job_param} \n \
    # s3_path {s3_path} \n \
    # classical_device {classical_device} \n \
    # image_uri {image_uri} ")

    tmp_job = AwsQuantumJob.create(
        device=quantum_device,
        source_module=f"{experiment_name}",
        entry_point=f"{experiment_name}.{job_name}:main",
        job_name=name,
        hyperparameters=job_param,
        input_data=s3_path,
        instance_config=InstanceConfig(instanceType=classical_device),
        image_uri=image_uri,
        wait_until_complete=False,
    )
    
    # tmp_job = LocalQuantumJob.create(
    #     device=quantum_device,
    #     source_module=f"{experiment_name}",
    #     entry_point=f"{experiment_name}.{job_name}:main",
    #     job_name=name,
    #     hyperparameters=job_param,
    #     input_data=s3_path,
    #     image_uri=image_uri,
    # )
    jobs.append(tmp_job)
    names.append(name)

fail to get null: list index out of range, use sv1 instead
Creating molecular-unfolding with M 1 and D 8


In [43]:
hybrid_jobs_json = f"{experiment_name}-hybrid-jobs.json"
print(f"job info will be saved in {hybrid_job_json}")

In [44]:
jobs_arn = []

for job in jobs:
    jobs_arn.append(job.arn)

jobs_states = {
    "experiment_name": experiment_name,
    "hybrid-jobs-arn": jobs_arn,
    "names": names
}


In [45]:
# save hybrid job arn for further analysis
json_object = json.dumps(jobs_states, indent=4)

with open(hybrid_jobs_json, "w") as outfile:
    outfile.write(json_object)

In [48]:
# recover hybrid jobs and show result
jobs_states_load = None
with open(hybrid_jobs_json, "r") as outfile:
    jobs_states_load = json.load(outfile)

for job_name, job_arn in zip(jobs_states_load["names"], jobs_states_load["hybrid-jobs-arn"]):
    current_job = AwsQuantumJob(job_arn)
    print(f"the state of job {job_name} is : {current_job.state()}")

the state of job molecular-unfolding-M-1-D-8-1672738111 is : COMPLETED
